# 🌺Lab 1

Klasifikasi Bunga Iris dengan Perceptron

## Langkah 1 - Import Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Langkah 2 - Load Data dan Visualisasi

In [ ]:
# Anda perlu memastikan file 'iris.csv' ada di direktori yang sama
# Jika menggunakan Google Colab, upload file tersebut
try:
    df = pd.read_csv('iris.csv', header=None)
except FileNotFoundError:
    print("File 'iris.csv' tidak ditemukan. Mengunduh dari URL...")
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
    df = pd.read_csv(url, header=None)
    df.to_csv('iris.csv', index=False, header=False)
    print("Dataset Iris telah diunduh dan disimpan sebagai 'iris.csv'.")

df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

setosa = df[df['species'] == 'Iris-setosa']
versicolor = df[df['species'] == 'Iris-versicolor']
virginica = df[df['species'] == 'Iris-virginica']

a, b = 'sepal_length', 'petal_length'
plt.scatter(setosa[a], setosa[b], color='red', marker='o', label='setosa')
plt.scatter(versicolor[a], versicolor[b], color='blue', marker='x', label='versicolor')

plt.xlabel('Sepal Length')
plt.ylabel('Petal Length')
plt.legend(loc='upper left')
plt.grid()
plt.show()

## Langkah 3 - Membuat Kelas Perceptron

In [ ]:
class Perceptron(object):
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter

    def fit(self, X, y):

        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

## Langkah 4 - Pilih Data dan Encoding Label

In [ ]:
y = df.iloc[0:100, 4].values # pilih 100 data awal (Setosa and Versicolor)
y = np.where(y == 'Iris-setosa', -1, 1) # ganti coding label
X = df.iloc[0:100, [0, 2]].values # slice data latih [sepal_length, petal_length]

## Langkah 5 - Fitting Model

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X, y)

## Langkah 6 - Visualisasi Nilai Error Per Epoch

In [ ]:
plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Number of updates')
plt.title('Perceptron - Learning Progress')
plt.grid()
plt.show()

## Langkah 7 - Visualisasi Decision Boundary

In [ ]:
# buat fungsi untuk plot decision region

from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.02):
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0],
                    y=X[y == cl, 1],
                    alpha=0.8,
                    c=colors[idx],
                    marker=markers[idx],
                    label='setosa' if cl == -1 else 'versicolor',
                    edgecolor='black')

### [Perbaikan] Memanggil Fungsi plot_decision_regions
Menambahkan visualisasi decision boundary yang sebelumnya belum dipanggil.

In [ ]:
plot_decision_regions(X, y, classifier=ppn)
plt.xlabel('Sepal length [cm]')
plt.ylabel('Petal length [cm]')
plt.legend(loc='upper left')
plt.title('Perceptron Decision Boundary')
plt.grid()
plt.show()

# 🌸Lab 2

Nilai Logika XOR dengan MLP

## Langkah 1 - Import Library

In [ ]:
from sklearn.neural_network import MLPClassifier

## Langah 2 - Buat Data

In [ ]:
y = [0, 1, 1, 0] # label
X = [[0, 0], [0, 1], [1, 0], [1, 1]] # data

## Langkah 3 - Fit Model

In [ ]:
# Fit model
# Menggunakan 'logistic' (sigmoid) sebagai fungsi aktivasi dan 'lbfgs' sebagai solver.
# hidden_layer_sizes=(2,) berarti ada satu hidden layer dengan 2 neuron.
# random_state digunakan untuk reproduktifitas hasil.
clf = MLPClassifier(solver='lbfgs', activation='logistic', hidden_layer_sizes=(2,), max_iter=1000, random_state=20)
clf.fit(X, y)

## Langkah 4 - Prediksi

In [ ]:
pred = clf.predict(X)
print('Accuracy: %s' % clf.score(X, y))
for i,p in enumerate(pred):
    print('True: %s, Predicted: %s' % (y[i], p))

#🌼Lab 3

Klasifikasi Churn Rate dengan ANN

## Pra Pengolahan Data

### Langkah 1 - Import Library

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Langkah 2 - Load Data

In [ ]:
# Pastikan file 'Churn_Modelling.csv' ada di direktori yang sama
try:
    dataset = pd.read_csv('Churn_Modelling.csv')
except FileNotFoundError:
    print("File 'Churn_Modelling.csv' tidak ditemukan. Coba unduh dari sumber lain...")
    # URL alternatif jika diperlukan, atau berikan pesan untuk mengupload manual
    print("Silakan upload file 'Churn_Modelling.csv' secara manual.")
    # Contoh: !wget [URL_FILE] # jika di Colab
    dataset = pd.DataFrame() # Buat dataframe kosong jika gagal

if not dataset.empty:
    X = dataset.iloc[:, 3:-1].values
    y = dataset.iloc[:, -1].values

In [ ]:
print(X)

### Langkah 3 - Encoding Data Kategorikal (Gender)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

### Langkah 4 - Encoding Kolom "Geography" dengan One Hot Encoder

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Langkah 5 - Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Langkah 6 - Scaling Fitur

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Membuat Model ANN

### Langkah 1 - Inisiasi Model ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Langkah 2 - Membuat Input Layer dan Hidden Layer Pertama

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Langkah 3 - Membuat Hidden Layer Kedua

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Langkah 4 - Membuat Output Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training Model

### Langkah 1 - Compile Model (Menyatukan Arsitektur) ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Langkah 2 - Fitting Model

In [ ]:
history_churn = ann.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_split=0.2)

### [Perbaikan] Visualisasi Hasil Training
Menambahkan plot untuk akurasi dan loss selama training untuk memantau performa model.

In [ ]:
pd.DataFrame(history_churn.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the y-axis range to [0-1]
plt.title('Churn Model Training History')
plt.xlabel('Epochs')
plt.show()

## Membuat Prediksi
Diberikan informasi sebagai berikut,

*   Geography: France
*   Credit Score: 600
*   Gender: Male
*   Age: 40 years old
*   Tenure: 3 years
*   Balance: $ 60000
*   Number of Products: 2
*   Does this customer have a credit card ? Yes
*   Is this customer an Active Member: Yes
*   Estimated Salary: $ 50000

Apakah customer tersebut akan Churn (keluar)?

**Penjelasan Data:**
- Geography 'France' di-encode menjadi `1, 0, 0`.
- Gender 'Male' di-encode menjadi `1`.
- HasCrCard 'Yes' adalah `1`.
- IsActiveMember 'Yes' adalah `1`.

## Modelkan Data Baru dan Buat Prediksi

In [ ]:
new_customer_data = [[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]
new_customer_scaled = sc.transform(new_customer_data)
prediction_prob = ann.predict(new_customer_scaled)
prediction = (prediction_prob > 0.5)

print(f"Data Customer Baru: {new_customer_data}")
print(f"Probabilitas Churn: {prediction_prob[0][0]:.4f}")
print(f"Prediksi Churn (Threshold 0.5): {prediction[0][0]}")

if prediction[0][0]:
    print("Hasil: Pelanggan diprediksi akan CHURN.")
else:
    print("Hasil: Pelanggan diprediksi TIDAK AKAN CHURN (akan tetap bertahan).")

## Prediksi Dengan Data Testing

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

## Cek Akurasi dan Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy Score: {acc:.4f}")

## 🌻Lab 4
Klasifikasi Siang dan Malam dengan ANN

## Langkah 1 - Import Library

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras import layers, models
import os
import zipfile
import requests

## Langkah 2 - Load Dataset

Kode ini akan mengunduh dataset jika folder `images` belum ada.

In [ ]:
if not os.path.exists('images'):
    print("Folder 'images' tidak ditemukan. Mengunduh dan mengekstrak dataset...")
    url = 'https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip'
    response = requests.get(url)
    with open('rockpaperscissors.zip', 'wb') as f:
        f.write(response.content)
    with zipfile.ZipFile('rockpaperscissors.zip', 'r') as zip_ref:
        zip_ref.extractall('images_temp')
    os.remove('rockpaperscissors.zip')

    # Strukturnya mungkin berbeda, kita perlu menyesuaikannya
    # Untuk contoh ini, kita asumsikan dataset siang/malam sudah ada
    # Jika tidak, Anda perlu menyediakannya dengan struktur:
    # images/training/day/*.jpg
    # images/training/night/*.jpg
    # images/test/day/*.jpg
    # images/test/night/*.jpg
    print("\nPERINGATAN: Dataset yang diunduh adalah 'rockpaperscissors'.")
    print("Pastikan Anda memiliki dataset 'day/night' dalam folder 'images' sesuai struktur.")

# Load images and labels from a directory structure
def load_dataset(img_dir):
    p = Path(img_dir)
    if not p.exists():
        print(f"Direktori tidak ditemukan: {img_dir}")
        return []
    img_list = []
    for folder in p.glob('*'):
        if not folder.is_dir(): continue
        label = folder.name
        for file in list(folder.glob('*.jpg')) + list(folder.glob('*.png')):
            img = cv2.imread(str(file))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_list.append((img, label))
    return img_list

train_dir = "images/training/"
test_dir  = "images/test/"

train_img = load_dataset(train_dir)
test_img  = load_dataset(test_dir)

if not train_img or not test_img:
    print("\nDataset tidak berhasil dimuat. Pastikan folder 'images/training' dan 'images/test' ada dan berisi gambar.")

## Langkah 2 - Pra Pengolahan

In [ ]:
# Preprocess images: resize and encode labels
def resize_image(img, size=(256,256)):
    return cv2.resize(img, size)

def label_encoder(label):
    return 1 if label.lower() == 'day' else 0

def preprocess(img_list):
    X = []
    y = []
    for img, label in img_list:
        img_std = resize_image(img)
        X.append(img_std)
        y.append(label_encoder(label))
    return X, y

if train_img and test_img:
    X_train_img, y_train = preprocess(train_img)
    X_test_img,  y_test  = preprocess(test_img)

## Langkah 3 - Ekstraksi Fitur

In [ ]:
# Extract HOG features
def extract_hog(X_imgs):
    feats = []
    for img in X_imgs:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        hog_feat = hog(gray,
                       orientations=9,
                       pixels_per_cell=(8,8),
                       cells_per_block=(2,2),
                       block_norm='L2-Hys',
                       visualize=False,
                       feature_vector=True)
        feats.append(hog_feat)
    return np.array(feats)

if train_img and test_img:
    X_train_feat = extract_hog(X_train_img)
    X_test_feat  = extract_hog(X_test_img)

## Langkah 4 - Standardisasi Fitur

In [ ]:
if train_img and test_img:
    # Normalize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_feat)
    X_test_scaled  = scaler.transform(X_test_feat)

## Langkah 5 - Buat Data Latih dan Validasi

In [ ]:
if train_img and test_img:
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train_scaled, y_train,
        test_size=0.2,
        random_state=42,
        stratify=y_train # Penting untuk data imbalanced
    )

    # Convert label ke numpy array
    y_train_split = np.array(y_train_split)
    y_val   = np.array(y_val)
    y_test  = np.array(y_test)

## Langkah 6 - Buat Model ANN

In [ ]:
if train_img and test_img:
    input_dim = X_train_split.shape[1]

    model_daynight = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5), # Tambah dropout untuk regularisasi
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])

    model_daynight.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model_daynight.summary()

## Langkah 7 - Latih Model

In [ ]:
if train_img and test_img:
    history_daynight = model_daynight.fit(
        X_train_split, y_train_split,
        epochs=20,
        batch_size=32,
        validation_data=(X_val, y_val)
    )

## Langkah 8 - Evaluasi Model dengan Data Test

In [ ]:
if train_img and test_img:
    test_loss, test_acc = model_daynight.evaluate(X_test_scaled, y_test)
    print(f"Akurasi Test: {test_acc:.4f}")

## Langkah 9 - Buat Laporan Performansi Model

In [ ]:
if train_img and test_img:
    y_pred_prob = model_daynight.predict(X_test_scaled)
    y_pred = (y_pred_prob > 0.5).astype(int)

    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['Night (0)', 'Day (1)']))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

## Langkah 10 - Visualisasi Proses Training

In [ ]:
if train_img and test_img:
    history_df = pd.DataFrame(history_daynight.history)
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))

    ax[0].plot(history_df['accuracy'], label='Train Acc')
    ax[0].plot(history_df['val_accuracy'], label='Val Acc')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Accuracy')
    ax[0].set_title('Training and Validation Accuracy')
    ax[0].legend()
    ax[0].grid(True)

    ax[1].plot(history_df['loss'], label='Train Loss')
    ax[1].plot(history_df['val_loss'], label='Val Loss')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Loss')
    ax[1].set_title('Training and Validation Loss')
    ax[1].legend()
    ax[1].grid(True)

    plt.show()

## Langkah 11 - Simpan Model

In [ ]:
if train_img and test_img:
    model_daynight.save('day_night_classifier_model.h5')
    print("Model berhasil disimpan sebagai 'day_night_classifier_model.h5'")

---# 🌹 Tugas Lab

**Tugas: Lakukan klasifikasi pada data MNIST dengan menggunakan model ANN.**

Anda diperbolehkan melakukan eksplorasi terhadap:
*   Metode pra pengolahan
*   Arsitektur ANN
*   Fungsi Aktivasi

> ANN diimplementasikan dengan menggunakan TensorFlow.

### Langkah 1: Import Library dan Memuat Dataset MNIST

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# Memuat dataset MNIST dari TensorFlow Keras
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(f"Ukuran data training (gambar): {x_train.shape}")
print(f"Ukuran data training (label): {y_train.shape}")
print(f"Ukuran data testing (gambar): {x_test.shape}")
print(f"Ukuran data testing (label): {y_test.shape}")

### Langkah 2: Pra-Pengolahan Data (Preprocessing)

In [ ]:
# Normalisasi: Mengubah nilai piksel dari rentang [0, 255] ke [0, 1]
# Ini membantu proses training menjadi lebih stabil dan cepat.
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flattening: Mengubah gambar 28x28 piksel menjadi vektor 1D berukuran 784
# Model Dense (fully connected) memerlukan input dalam bentuk vektor.
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

print(f"Ukuran data training setelah flatten: {x_train_flat.shape}")
print(f"Contoh label: {y_train[0]}")
print(f"Contoh data piksel (setelah normalisasi): {x_train[0][10, 10:15]}")

### Langkah 3: Visualisasi Contoh Data MNIST

In [ ]:
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
plt.suptitle("Contoh Gambar dari Dataset MNIST")
plt.show()

### Langkah 4: Membangun Arsitektur Model ANN

Arsitektur yang akan kita bangun:
1.  **Input Layer**: Berupa layer `Flatten` untuk mengubah matriks 28x28 menjadi vektor 784.
2.  **Hidden Layer 1**: `Dense` layer dengan 128 neuron dan fungsi aktivasi `ReLU`.
3.  **Dropout Layer**: Untuk regularisasi, mencegah overfitting dengan menonaktifkan beberapa neuron secara acak selama training.
4.  **Hidden Layer 2**: `Dense` layer dengan 64 neuron dan fungsi aktivasi `ReLU`.
5.  **Output Layer**: `Dense` layer dengan 10 neuron (karena ada 10 kelas, 0-9) dan fungsi aktivasi `Softmax`. Softmax akan memberikan probabilitas untuk setiap kelas.

In [ ]:
model_mnist = tf.keras.models.Sequential([
    # Tidak perlu flatten layer jika input sudah di-flatten manual
    # tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Input(shape=(784,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile model
model_mnist.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Cocok untuk label integer
              metrics=['accuracy'])

### Langkah 5: Melatih Model (Training)

In [ ]:
history_mnist = model_mnist.fit(x_train_flat, y_train, epochs=10, validation_split=0.2)

### Langkah 6: Evaluasi Model

In [ ]:
test_loss, test_acc = model_mnist.evaluate(x_test_flat, y_test, verbose=2)
print(f'\nAkurasi pada data test: {test_acc:.4f}')

### Langkah 7: Visualisasi Hasil Training

In [ ]:
history_df = pd.DataFrame(history_mnist.history)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].plot(history_df['accuracy'], label='Train Acc')
ax[0].plot(history_df['val_accuracy'], label='Val Acc')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Training and Validation Accuracy')
ax[0].legend()
ax[0].grid(True)

ax[1].plot(history_df['loss'], label='Train Loss')
ax[1].plot(history_df['val_loss'], label='Val Loss')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].set_title('Training and Validation Loss')
ax[1].legend()
ax[1].grid(True)

plt.show()

### Langkah 8: Analisis Hasil Prediksi (Confusion Matrix dan Laporan Klasifikasi)

In [ ]:
y_pred_prob = model_mnist.predict(x_test_flat)
y_pred = np.argmax(y_pred_prob, axis=1) # Mengambil kelas dengan probabilitas tertinggi

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for MNIST Classification')
plt.show()